In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [106]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

Rewrite labs to get closest lab, if more than one lab, average. 

In [38]:
query = \
"""
SELECT * FROM echo_features_labs; 
"""
all_labs = pd.read_sql_query(query, con)
all_labs.head()

,row_id,subject_id,hadm_id,icustay_id,abs_dt,label,valuenum
0,67327,21,109451,217847,1 days 21:22:00,WBC,13.00
1,67327,21,109451,217847,1 days 15:52:00,CK-MB,16.20
2,67327,21,109451,217847,1 days 15:52:00,TROPT,3.02
3,67327,21,109451,217847,1 days 15:29:00,HEMATOCRIT,27.00
4,67327,21,109451,217847,1 days 15:29:00,HEMOGLOBIN,9.00


In [39]:
# check that only 1 observed value per (row_id, label, abs_dt)
all_labs.groupby(['row_id', 'label', 'abs_dt'])['valuenum'].count().value_counts()

1    4182610
2       5304
0         26
3         15
Name: valuenum, dtype: int64

In [41]:
query = \
"""
SELECT * FROM echo_features_labs; 
"""
squashed_labs = pd.read_sql_query(query, con)
squashed_labs.head()

,row_id,label,abs_dt,valuenum
0,85531,CREATININE,3 days 15:04:00,0.9
1,90694,CREATININE,2 days 09:17:00,1.7
2,91594,HEMOGLOBIN,0 days 02:16:00,7.9
3,66276,CREATININE,1 days 11:31:00,0.6
4,89890,HEMATOCRIT,10 days 16:50:00,25.9


In [57]:
len(squashed_labs['row_id'].unique())

25512

Some subjects have no data:

In [61]:
squashed_labs.loc[squashed_labs['label'].isnull()].shape

(104, 4)

In [62]:
squashed_labs.loc[squashed_labs['label'].isnull()].head()

,row_id,label,abs_dt,valuenum
16547,94453,None,0 days,NaN
21927,62614,None,0 days,NaN
50207,100460,None,0 days,NaN
67172,62276,None,0 days,NaN
164157,71520,None,0 days,NaN


In [42]:
# check that only 1 observed value per (row_id, label, abs_dt)
squashed_labs.groupby(['row_id', 'label', 'abs_dt'])['valuenum'].count().value_counts()

1    4187929
0         26
Name: valuenum, dtype: int64

In [44]:
closest_labs = squashed_labs.sort_values('abs_dt').groupby(['row_id', 'label']).first()['valuenum']
closest_labs.unstack('label').head()

label,ALBUMIN,BICARBONATE,CK-MB,CREATININE,CRP,HEMATOCRIT,HEMOGLOBIN,INR,LACTATE,NTPROBNP,PH,PLATELET,TROPI,TROPT,WBC
row_id,,,,,,,,,,,,,,,
59653,NaN,21.0,4.6,0.8,NaN,32.8,11.8,1.0,4.0,NaN,NaN,155.0,NaN,0.02,7.4
59654,4.1,26.0,NaN,1.6,NaN,33.1,11.0,1.1,1.6,4453.0,7.39,219.0,NaN,NaN,8.6
59655,3.4,25.0,NaN,0.7,NaN,34.2,11.7,2.4,1.1,NaN,7.45,212.0,NaN,NaN,10.1
59656,2.2,30.0,NaN,3.3,NaN,32.4,9.6,1.3,2.0,NaN,7.49,153.0,NaN,0.25,11.8
59657,2.6,27.0,1.4,3.3,NaN,28.0,9.4,1.9,4.5,NaN,7.45,66.0,NaN,0.11,7.9


In [53]:
len(closest_labs.dropna().reset_index()['row_id'].unique())

25408

In [45]:
query = \
"""
SELECT * FROM echo_features_labs; 
"""
summary_labs = pd.read_sql_query(query, con)
summary_labs.loc[summary_labs['row_id'] == 59653]

,row_id,lab_albumin,lab_bicarbonate,lab_ckmb,lab_creatinine,lab_crp,lab_egfr,lab_hematocrit,lab_hemoglobin,lab_inr,lab_lactate,lab_platelet,lab_ntprobnp,lab_ph,lab_tropi,lab_tropt,lab_wbc
7959,59653,NaN,21.0,4.6,0.8,NaN,None,32.8,11.8,1.0,4.0,155.0,NaN,NaN,NaN,0.02,7.4


In [29]:
all_labs.loc[(all_labs['row_id'] == 59653) & (all_labs['label'] == 'WBC')].sort_values('abs_dt')

,row_id,subject_id,hadm_id,icustay_id,label,abs_dt,valuenum
3009633,59653,31038,174978,200492,WBC,0 days 01:50:00,7.4
916478,59653,31038,174978,200492,WBC,0 days 10:18:00,7.1
2363687,59653,31038,174978,200492,WBC,0 days 16:40:00,6.8
862270,59653,31038,174978,200492,WBC,1 days 12:11:00,8.9
59358,59653,31038,174978,200492,WBC,1 days 17:00:00,7.2
3636386,59653,31038,174978,200492,WBC,2 days 05:30:00,16.9
1832459,59653,31038,174978,200492,WBC,2 days 11:35:00,20.3


Implement filters. 

In [104]:
query = \
"""
SELECT * FROM echo_filtered; 
"""
filtered = pd.read_sql_query(query, con)
filtered.head()

,row_id,icustay_id,hadm_id,subject_id,age_at_intime,gender,height,weight,bmi,ethnicity,...,fl_day1_input_ml,fl_day1_output_ml,fl_day1_balance_ml,fl_day2_input_ml,fl_day2_output_ml,fl_day2_balance_ml,fl_day3_input_ml,fl_day3_output_ml,fl_day3_balance_ml,most_recent_hadm
0,67326,216859,111970,21,32051 days 20:53:34,M,73.0,166.0,21.898668,WHITE,...,2606.25,129.0,2477.25,1267.5,48.0,1219.5,1750.0,40.0,1710.0,1
1,67805,280987,182104,36,25443 days 10:29:14,M,NaN,NaN,NaN,WHITE,...,3001.00,1885.0,1116.00,508.0,1201.0,-693.0,NaN,NaN,NaN,1
2,67118,216609,116009,62,25097 days 00:20:44,M,68.0,150.0,22.804931,PATIENT DECLINED TO ANSWER,...,2781.00,698.0,2083.00,505.0,1525.0,-1020.0,335.5,730.0,-394.5,1
3,80320,281607,112086,75,28018 days 10:35:50,F,NaN,NaN,NaN,WHITE,...,3040.00,1410.0,1630.00,12.0,1475.0,-1463.0,NaN,NaN,NaN,1
4,80083,206222,190243,86,16521 days 11:51:54,F,NaN,NaN,NaN,WHITE,...,3520.00,1707.0,1813.00,1006.0,1430.0,-424.0,NaN,NaN,NaN,1


In [200]:
# hospital admission ids match
set(most_recent_hadm['hadm_id'].unique()) - set(uniq['hadm_id'].unique())

set()

In [230]:
most_recent_hadm_ids = filtered.loc[filtered.groupby('subject_id').apply(lambda x: x['admittime'].idxmax()), 'hadm_id'].un
most_recent_hadm = filtered.loc[filtered['hadm_id'].apply(lambda i: i in most_recent_hadm_ids)]

first_echo = most_recent_hadm.loc[most_recent_hadm.groupby('subject_id').apply(lambda x: x['ed_charttime'].idxmin())]

check_ids = set(first_echo['row_id'].unique())

first_echo.head()

ValueError: Cannot index with multidimensional key

In [228]:
for subject_id, group in most_recent_hadm.sort_values('ed_charttime').groupby('subject_id'):
    if subject_id == 16751:
        break
group

,row_id,icustay_id,hadm_id,subject_id,age_at_intime,gender,height,weight,bmi,ethnicity,...,fl_day1_input_ml,fl_day1_output_ml,fl_day1_balance_ml,fl_day2_input_ml,fl_day2_output_ml,fl_day2_balance_ml,fl_day3_input_ml,fl_day3_output_ml,fl_day3_balance_ml,most_recent_hadm
519,64129,245210,181269,16751,15119 days 12:42:07,M,NaN,NaN,NaN,WHITE,...,5625.0,2290.0,3335.0,1004.0,1904.0,-900.0,NaN,NaN,NaN,1


In [204]:
query = \
"""
SELECT * FROM echo_filtered; 
"""
uniq = pd.read_sql_query(query, con)
final_ids = set(uniq['row_id'])
print(uniq.shape)
uniq.head()

(2189, 114)


,row_id,icustay_id,hadm_id,subject_id,age_at_intime,gender,height,weight,bmi,ethnicity,...,fl_day1_output_ml,fl_day1_balance_ml,fl_day2_input_ml,fl_day2_output_ml,fl_day2_balance_ml,fl_day3_input_ml,fl_day3_output_ml,fl_day3_balance_ml,most_recent_hadm,first_echo
0,67326,216859,111970,21,32051 days 20:53:34,M,73.0,166.0,21.898668,WHITE,...,129.0,2477.25,1267.5,48.0,1219.5,1750.0,40.0,1710.0,1,1
1,67805,280987,182104,36,25443 days 10:29:14,M,NaN,NaN,NaN,WHITE,...,1885.0,1116.00,508.0,1201.0,-693.0,NaN,NaN,NaN,1,1
2,67118,216609,116009,62,25097 days 00:20:44,M,68.0,150.0,22.804931,PATIENT DECLINED TO ANSWER,...,698.0,2083.00,505.0,1525.0,-1020.0,335.5,730.0,-394.5,1,1
3,80320,281607,112086,75,28018 days 10:35:50,F,NaN,NaN,NaN,WHITE,...,1410.0,1630.00,12.0,1475.0,-1463.0,NaN,NaN,NaN,1,1
4,80083,206222,190243,86,16521 days 11:51:54,F,NaN,NaN,NaN,WHITE,...,1707.0,1813.00,1006.0,1430.0,-424.0,NaN,NaN,NaN,1,1


In [205]:
list(check_ids - final_ids)[:5]

[81280, 64129, 84738, 82821, 99975]

In [213]:
filtered.loc[filtered['row_id'] == 64129]

,row_id,icustay_id,hadm_id,subject_id,age_at_intime,gender,height,weight,bmi,ethnicity,...,fl_day1_input_ml,fl_day1_output_ml,fl_day1_balance_ml,fl_day2_input_ml,fl_day2_output_ml,fl_day2_balance_ml,fl_day3_input_ml,fl_day3_output_ml,fl_day3_balance_ml,most_recent_hadm
519,64129,245210,181269,16751,15119 days 12:42:07,M,NaN,NaN,NaN,WHITE,...,5625.0,2290.0,3335.0,1004.0,1904.0,-900.0,NaN,NaN,NaN,1


In [214]:
filtered.loc[filtered['subject_id'] == 16751, ['row_id', 'subject_id', 'admittime', 'ed_charttime']]

,row_id,subject_id,admittime,ed_charttime
519,64129,16751,2126-07-25 12:40:00,2126-07-26 10:38:00
520,64130,16751,2126-07-25 12:40:00,2126-07-25 16:17:00


In [217]:
filtered.loc[filtered['subject_id'] == 16751, 'ed_charttime'].idxmin()

520

In [215]:
uniq.loc[uniq['subject_id'] == 16751, ['row_id', 'subject_id', 'admittime', 'ed_charttime']]

,row_id,subject_id,admittime,ed_charttime
479,64130,16751,2126-07-25 12:40:00,2126-07-25 16:17:00
